## [무작정 kaggle 따라하기] 코로나-19 대한민국 분석

*kaggle 연습을 위한 notebook입니다. <br>
*해당 커널은 다음 자료를 참고하였습니다. <br>
* [vanshjatana의 Analysis on Coronavirus](https://www.kaggle.com/vanshjatana/analysis-on-coronavirus)

![사회적 거리두기](https://www.gyeongju.go.kr/upload/ckuploads/2020/20200405/08FCA747201F4969994E87EA07D6D25C.png)

코로나-19, 현재까지 한국에서 어떠한 양상으로 진행되고 있는 것일까? <br>
정부는 연일 '사회적 거리두기'를 강조하며 사람들간의 접촉을 자제하라고 당부하고 있다. <br>
코로나-19에 대해 아직 안심하기는 시기상조인지 환자 데이터를 통해 알아보고자 한다!

### 라이브러리 불러오기

In [ ]:
import numpy as np # 선형대수
import pandas as pd # 데이터 프로세싱 (csv 파일 읽기 등)
import seaborn as sns # 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px

from datetime import date, timedelta
from sklearn.cluster import KMeans # k 평균 군집
from fbprophet import Prophet # 시계열 예측
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py
from statsmodels.tsa.arima_model import ARIMA # arima 시계열
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from keras.models import Sequential # 딥러닝 케라스
from keras.layers import LSTM,Dense
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# kaggle directory 에서 데이터 불러오기
# coronavirusdataset 사용
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### 코로나-19 증상 분류하기 <br>
[해당 문서 참고](https://en.wikipedia.org/wiki/Coronavirus_disease_2019)

In [ ]:
# 증상 분류 (증상 별 발현 퍼센테이지 담은 data frame 만들기)
symptoms={'symptom':['Fever',
        'Dry cough',
        'Fatigue',
        'Sputum production',
        'Shortness of breath',
        'Muscle pain',
        'Sore throat',
        'Headache',
        'Chills',
        'Nausea or vomiting',
        'Nasal congestion',
        'Diarrhoea',
        'Haemoptysis',
        'Conjunctival congestion'],'percentage':[87.9,67.7,38.1,33.4,18.6,14.8,13.9,13.6,11.4,5.0,4.8,3.7,0.9,0.8]}

symptoms=pd.DataFrame(data=symptoms,index=range(14))
symptoms

In [ ]:
# bar chart로 나타내기
fig = px.bar(symptoms[['symptom','percentage']].sort_values('percentage', ascending=False),
            y='percentage', x='symptom', color='symptom',
            log_y=True, template='ggplot2', title='Symptoms of Coronavirus')
fig.show()

In [ ]:
# pie chart로 나타내기
fig = px.pie(symptoms,
             values='percentage',
             names='symptom',
             template='seaborn')
fig.update_traces(rotation=90, pull=0.05, textinfo='percent+label')
fig.show()

In [ ]:
# tree plot으로 나타내기
fig = px.treemap(symptoms, path=['symptom'], values='percentage',
                color='percentage', hover_data=['symptom'],
                color_continuous_scale='Rainbow')
fig.show()

In [ ]:
# word cloud로 나타내기
from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in symptoms.symptom)
wordcloud = WordCloud(max_words=200, colormap='Set3', background_color='white').generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1, figsize=(12,12))
plt.show()

### 환자 데이터 분포 확인하기

In [ ]:
comp = pd.read_excel('/kaggle/input/covid19327/COVID-19-3.27-top30-500.xlsx')

df_patient = pd.read_csv('../input/patient/patient.csv')
df_route = pd.read_csv('../input/route/route.csv')

weather = pd.read_csv('../input/coronavirusdataset/Weather.csv')

In [ ]:
# 환자 데이터 분류 알아보기
df_patient.head()

1. id : 환자 ID
2. sex : 성별
3. birth_year : 출생연도
4. country : 국가
5. region : 세부 지역
6. group : 집단 감염 여부
7. infection_reason : 감염 사유
8. infection_order : 감염 여부
9. infected_by : 어떤 환자 ID로부터 감염되었는지
10. contact_number : 접촉자 수
11. confirmed_date : 확진일
12. released_date : 증상 발현일
13. deceased_date : 사망일
14. state : 현재 상태 (isolated 격리 / released 격리해제 / deceased 사망)

In [ ]:
df_patient.dtypes

### 결측치 확인 및 처리 & 변수 생성

In [ ]:
# 결측치 총합 나타내기
df_patient.isna().sum()

In [ ]:
# 출생연도 결측치 처리 : 정의할 수 없는 수(NaN)으로 치환
df_patient['birth_year'] = df_patient.birth_year.fillna(0.0).astype(int)
df_patient['birth_year'] = df_patient['birth_year'].map(lambda val: val if val > 0 else np.nan)

In [ ]:
# 확진일 datetime으로 type 변경하기
df_patient.confirmed_date = pd.to_datetime(df_patient.confirmed_date)
# daily_count (하루에 발생한 확진자 수 확인 변수) 생성하기
daily_count = df_patient.groupby(df_patient.confirmed_date).id.count()
# accumulated_count (누적 확진자 수 확인 변수) 생성하기
accumulated_count = daily_count.cumsum()

In [ ]:
# 나이 변수 만들기
df_patient['age'] = 2020 - df_patient['birth_year']
df_patient['age'].head()

In [ ]:
# 나이 분포 분류하기
import math
def group_age(age):
    if age >= 0: # NaN 아닐 때
        if age % 10 != 0: # 10대가 아닐 때
            lower = int(math.floor(age / 10.0)) * 10
            upper = int(math.ceil(age / 10.0)) * 10 - 1
            return f"{lower}-{upper}"
        else: # 10세 이하 영유아
            lower = int(age)
            upper = int(age+9)
            return f"{lower}-{upper}"
    return "Unknown" # NaN 일 때

df_patient['age_range'] = df_patient['age'].apply(group_age)
df_patient['age_range'].head()

In [ ]:
# df_patient를 사용하기 쉽게 patient로 변경함
patient = df_patient

In [ ]:
# 날짜 columns datetime으로 type 변환
date_cols = ['confirmed_date', 'released_date', 'deceased_date']
for col in date_cols:
    patient[col] = pd.to_datetime(patient[col])

In [ ]:
# time_to_release_since_confirmed (확정 이후로 증상 발현 시작한 날짜) 변수 만들기
patient['time_to_release_since_confirmed'] = patient['released_date'] - patient['confirmed_date']
# time_to_death_since_confirmed (확정 이후로 사망한 날짜) 변수 만들기
patient['time_to_death_since_confirmed'] = patient['deceased_date'] - patient['confirmed_date']
# duration_since_confirmed (확정 이후 지속 기간) 변수 만들기
patient['duration_since_confirmed'] = patient[['time_to_release_since_confirmed', 'time_to_death_since_confirmed']].min(axis=1)
# duration_days (확정 이후 지속 기간 일수) 변수 만들기
patient['duration_days'] = patient['duration_since_confirmed'].dt.days

# age range 분류
age_ranges = sorted(set([ar for ar in patient['age_range'] if ar != 'Unknown']))
# state_by_gender (성별과 현재 상태) 변수 만들기
patient['state_by_gender'] = patient['state'] + '-' + patient['sex']

### 누적 확진자수 그래프로 나타내기

In [ ]:
accumulated_count.plot()
plt.title('Accumulated Confirmed Count')

2020년 2월 20일 부근에서 시작하여 누적 확진자 수가 기하급수적으로 늘어나기 시작하였다. <br>
해당 확진자 수 증가는 ID 31번 대구 신천지 교인 환자가 '슈퍼 전파자' 역할을 하면서 위와 같이 확산된 것으로 보인다는 기사가 수차례 나온 바 있다. <br>

*[관련 기사 참고: 신천지 대구교회서 '슈퍼전파' 발생…예배 참석자 전원조사(종합2보) / 2020.02.19](https://www.yna.co.kr/view/AKR20200219114353017?section=search)*

### 시계열 분석 (최근 2주 데이터만 불러오기)

In [ ]:
# 최근 2주 대한민국 데이터 불러오기
time = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
time = time[time['Country/Region'] == 'South Korea']
time = time.tail(14)

### 누적 확진 / 회복 / 사망자 수 분포 정리

In [ ]:
# bar chart - 최근 2주 누적수 분포 시각화
plt.figure(figsize=(23, 10))
plt.bar(time.ObservationDate, time.Confirmed, label='Confirm')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend(frameon=True, fontsize=12)
plt.title('Confirm', fontsize=30)
plt.show()

plt.figure(figsize=(23, 10))
plt.bar(time.ObservationDate, time.Recovered, label='Recovery')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend(frameon=True, fontsize=12)
plt.title('Recovery', fontsize=30)
plt.show()

plt.figure(figsize=(23, 10))
plt.bar(time.ObservationDate, time.Deaths, label='Death')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend(frameon=True, fontsize=12)
plt.title('Death', fontsize=30)
plt.show()

In [ ]:
# bar chart - 최근 2주 누적수 한 그래프에 보이게 시각화 하기
plt.figure(figsize=(23, 10))
plt.bar(time.ObservationDate, time.Confirmed, label='Confirm')
plt.bar(time.ObservationDate, time.Recovered, label='Recovery')
plt.bar(time.ObservationDate, time.Deaths, label='Death')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend(frameon=True, fontsize=12)
plt.title('Confirm vs Recovery vs Death', fontsize=30)
plt.show()

# line plot - 최근 2주 누적수 점 분포로 나타내기
f, ax = plt.subplots(figsize=(23,10))
ax = sns.scatterplot(x='ObservationDate', y='Confirmed', data=time, color='black', label='Confirm')
ax = sns.scatterplot(x='ObservationDate', y='Recovered', data=time, color='red', label='Recovery')
ax = sns.scatterplot(x='ObservationDate', y='Deaths', data=time, color='blue', label='Death')
plt.plot(time.ObservationDate, time.Confirmed, zorder=1, color='black')
plt.plot(time.ObservationDate, time.Recovered, zorder=1, color='red')
plt.plot(time.ObservationDate, time.Deaths, zorder=1, color='blue')

전체 비율을 볼 때, 회복자수는 상승하는 반면 확진자수와 사망자수는 더디게 상승하고 있어 긍정적으로 전망할 수 있다.

In [ ]:
# pie chart - 최근 2주 총 확진/회복/사망자수 파이 그래프로 나타내기
total_confirmed = time['Confirmed'].sum()
total_recovered = time['Recovered'].sum()
total_death = time['Deaths'].sum()

data = [['Confirmed', total_confirmed], ['Recovered', total_recovered], ['Death', total_death]]
df = pd.DataFrame(data, columns = ['state', 'count'])
fig = px.pie(df,
            values='count',
            names='state',
            title='State of Patient',
            template='seaborn')
fig.update_traces(rotation=90, pull=0.05, textinfo='percent+label')
fig.show()

pie chart에서는 현재 확진자수와 회복자수의 비율이 거의 비슷해지고 있는 상황이라 앞서 살펴본 그래프를 더해볼 때, <br>
회복자수와 확진자수의 증가 추이가 위와 같이 흘러간다면 회복자수의 비율이 확진자수의 비율보다 앞설 것으로 예상할 수 있다.

### 일일 증가율 살펴보기

In [ ]:
# 변수 생성
time['Confirmed_new'] = time['Confirmed']-time['Confirmed'].shift(1)
time['Recovered_new'] = time['Recovered']-time['Recovered'].shift(1)
time['Deaths_new'] = time['Deaths']-time['Deaths'].shift(1)

In [ ]:
# bar chart - 일일 증가수 시각화
plt.figure(figsize=(23,10))
plt.bar(time.ObservationDate, time.Confirmed_new,label="Confirm")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Confirm',fontsize=30)
plt.show()

plt.figure(figsize=(23,10))
plt.bar(time.ObservationDate, time.Recovered_new,label="Recovery")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Recovery',fontsize=30)
plt.show()

plt.figure(figsize=(23,10))
plt.bar(time.ObservationDate, time.Deaths_new,label="Death")
plt.xlabel('Date')
plt.ylabel("Count")
plt.legend(frameon=True, fontsize=12)
plt.title('Death',fontsize=30)
plt.show()

In [ ]:
# line plot - 일일 증가수 시각화
f, ax = plt.subplots(figsize=(23,10))
ax = sns.scatterplot(x='ObservationDate', y='Confirmed_new', data=time, color='black', label='Confirm')
ax = sns.scatterplot(x='ObservationDate', y='Recovered_new', data=time, color='red', label='Recovery')
ax = sns.scatterplot(x='ObservationDate', y='Deaths_new', data=time, color='blue', label='Death')
plt.plot(time.ObservationDate, time.Confirmed_new, zorder=1, color='black')
plt.plot(time.ObservationDate, time.Recovered_new, zorder=1, color='red')
plt.plot(time.ObservationDate, time.Deaths_new, zorder=1, color='blue')

In [ ]:
global_data = time

### 현재 상태별 증가율과 비율 나타내기

In [ ]:
def smoother(inputdata,w,imax):
    data = 1.0*inputdata
    data = data.replace(np.nan,1)
    data = data.replace(np.inf,1)
    #print(data)
    smoothed = 1.0*data
    normalization = 1
    for i in range(-imax,imax+1):
        if i==0:
            continue
        smoothed += (w**abs(i))*data.shift(i,axis=0)
        normalization += w**abs(i)
    smoothed /= normalization
    return smoothed

def growth_factor(confirmed): # 증가율
    confirmed_iminus1 = confirmed.shift(1, axis=0)
    confirmed_iminus2 = confirmed.shift(2, axis=0)
    return (confirmed-confirmed_iminus1)/(confirmed_iminus1-confirmed_iminus2)

def growth_ratio(confirmed): # 차지 비율
    confirmed_iminus1 = confirmed.shift(1, axis=0)
    return (confirmed/confirmed_iminus1)

# input에 있는 국가
def plot_country_active_confirmed_recovered(country):
    # 케이스 나누기
    country_data = global_data[global_data['Country/Region']==country]
    table = country_data.drop(['SNo', 'Province/State', 'Last Update'], axis=1)
    table['ActiveCases'] = table['Confirmed'] - table['Recovered'] - table['Deaths']
    table2 = pd.pivot_table(table, values=['ActiveCases', 'Confirmed', 'Recovered', 'Deaths'], index=['ObservationDate'], aggfunc=np.sum)
    table3 = table2.drop(['Deaths'], axis=1)
    
    # 증가율
    w = 0.5
    table2['GrowthFactor'] = growth_factor(table2['Confirmed'])
    table2['GrowthFactor'] = smoother(table2['GrowthFactor'],w,5)

    # 2차 미분
    table2['2nd_Derivative'] = np.gradient(np.gradient(table2['Confirmed']))
    table2['2nd_Derivative'] = smoother(table2['2nd_Derivative'],w,7)


    # 비율
    table2['GrowthRatio'] = growth_ratio(table2['Confirmed'])
    table2['GrowthRatio'] = smoother(table2['GrowthRatio'],w,5)
    
    # 로지스틱 회귀
    table2['GrowthRate']=np.gradient(np.log(table2['Confirmed']))
    table2['GrowthRate'] = smoother(table2['GrowthRate'],0.5,3)
    
    # 1.0부터 그래프로 나타냄
    x_coordinates = [1, 100]
    y_coordinates = [1, 1]
    f, ax = plt.subplots(figsize=(15,5))
    table2['Deaths'].plot(title='Deaths') # 일일 사망수 그래프
    plt.show()
    f, ax = plt.subplots(figsize=(15,5)) 
    table2['GrowthFactor'].plot(title='Growth Factor') # 증가율
    plt.plot(x_coordinates, y_coordinates) 
    plt.show()
    f, ax = plt.subplots(figsize=(15,5)) 
    table2['2nd_Derivative'].plot(title='2nd_Derivative') # 2차 미분
    plt.show()
    f, ax = plt.subplots(figsize=(15,5))
    table2['GrowthRatio'].plot(title='Growth Ratio') # 증가 비율
    plt.plot(x_coordinates, y_coordinates)
    plt.show()
    f, ax = plt.subplots(figsize=(15,5))
    table2['GrowthRate'].plot(title='Growth Rate') # 로지스틱 회귀
    plt.show()

    return 

In [ ]:
plot_country_active_confirmed_recovered('South Korea')

### 환자 현재 상태 분포 알아보기

In [ ]:
# 감염 환자
infected_patient = patient.shape[0]

# 발현 / 사망 / 격리로 분류
rp = patient.loc[patient["state"] == "released"].shape[0]
dp = patient.loc[patient["state"] == "deceased"].shape[0]
ip = patient.loc[patient["state"]== "isolated"].shape[0]

# 발현 / 사망 / 격리자 수 비율
rp=rp/patient.shape[0]
dp=dp/patient.shape[0]
ip=ip/patient.shape[0]

print("The percentage of recovery is "+ str(rp*100) )
print("The percentage of deceased is "+ str(dp*100) )
print("The percentage of isolated is "+ str(ip*100) )

In [ ]:
states = pd.DataFrame(patient["state"].value_counts())
states["status"] = states.index
states.rename(columns={"state": "count"}, inplace=True)
# 상태 별 인원수 세기

# pie chart - 발현 / 사망 / 격리자 수 비율 시각화
fig = px.pie(states,
             values="count",
             names="status",
             title="Current state of patients",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="value+percent+label")
fig.show()


In [ ]:
# 발현자 정보
released = df_patient[df_patient.state == 'released']
released.head()

In [ ]:
# 격리자 정보
isolated = df_patient[df_patient.state == 'isolated']
isolated.head()

In [ ]:
# 사망자 정보
deceased = df_patient[df_patient.state == 'deceased']
deceased.head()

### 발현 / 격리 / 사망자 별 나이 분포 알아보기

In [ ]:
# 발현자 나이 분포
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the released")
sns.kdeplot(data=released['age'], shade=True)

In [ ]:
agr = released[released.age_range!="Unknown"]
fig = px.pie(agr,
             names="age_range",
             title="Age of released person",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

발현자는 30대가 25.7%로 가장 많았고, 그 뒤를 이어 40-50대, 20대, 60대, 70대 순으로 나타났다. <br>
주로 청장년층에서 많이 발현되었고, 활발한 사회생활로 인하여 타인 접촉 빈도가 높기 때문으로 추정한다.

In [ ]:
# 격리자 나이 분포
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the isolated")
sns.kdeplot(data=isolated['age'], shade=True)

In [ ]:
agi = isolated[isolated.age_range!="Unknown"]
fig = px.pie(agi,
             names="age_range",
             title="Age of isolated person",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

격리자는 연령대별로 다양하게 분포되어 있다. 20대가 가장 많고, 그 이후로 50대, 30대와 60대가 뒤를 잇는다.

In [ ]:
# 사망자 나이 분포
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the deceased")
sns.kdeplot(data=deceased['age'], shade=True)

In [ ]:
agd = deceased[deceased.age_range!="Unknown"]
fig = px.pie(agd,
             names="age_range",
             title="Age of Dead person",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

사망자는 우리가 기존에 예상하고 있던 대로 고령층이 많은 비율을 차지하고 있다. 사망자의 절반이 60-70대이다. <br>
영유아기 ~ 20대 발현자에게서는 사망자 수가 나오지 않았다.

### 성별에 따른 사망자 나이별 분포 알아보기

In [ ]:
male_dead = deceased[deceased.sex=='male']
female_dead = deceased[deceased.sex=='female']

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Age distribution of the deceased by gender")
sns.kdeplot(data=female_dead['age'], label="Women", shade=True)
sns.kdeplot(data=male_dead['age'],label="Male" ,shade=True)

In [ ]:
plt.figure(figsize=(10,8))
sns.set_style("darkgrid")
sns.distplot(a=male_dead['age'], label="Men", kde=False)
sns.distplot(a=female_dead['age'], label="Women", kde=False)
plt.title("Age distribution of the deceased by sex")
plt.legend()

나이가 많을 수록 여성보다 남성의 사망자 수가 더 높다고 볼 수 있다.

### 나이에 따른 발현자와 사망자 수 비교하기

In [ ]:
sns.kdeplot(data=deceased['age'],label='deceased', shade=True)
sns.kdeplot(data=released['age'],label='released', shade=True)
sns.kdeplot(data=isolated['age'],label='isolated', shade=True)

나이가 어릴수록 격리자 분포가 높고, 나이가 많을 수록 사망자 분포가 높다.

### 성별에 따른 발현 / 격리 / 사망자 수 알아보기

In [ ]:
# 성별에 따른 사망자 수 
plt.figure(figsize=(15, 5))
plt.title('Sex')
deceased.sex.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=deceased.groupby(['sex']).size().values,names=deceased.groupby(['sex']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

남성이 여성보다 사망자 수 비율이 2배 이상으로 나타난다.

In [ ]:
# 성별에 따른 격리자 수
plt.figure(figsize=(15, 5))
plt.title('Sex')
isolated.sex.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=isolated.groupby(['sex']).size().values,names=isolated.groupby(['sex']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

격리자 수는 성별에 따른 차이가 크게 없다.

In [ ]:
# 성별에 따른 발현자 수
plt.figure(figsize=(15, 5))
plt.title('Sex')
released.sex.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=released.groupby(['sex']).size().values,names=released.groupby(['sex']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

발현자 수 또한 성별에 따른 차이가 크게 보이지 않는다.

### 감염 사유 알아보기

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Infection Reason')
df_patient.infection_reason.value_counts().plot.bar();

In [ ]:
fig = px.pie( values=df_patient.groupby(['infection_reason']).size().values,names=df_patient.groupby(['infection_reason']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

감염 사유는 환자와의 접촉이 절반으로 가장 많았고, 그 뒤로 대구나 우한과 같은 폭발적 발현 지역 방문, 이스라엘이나 싱가포르와 같은 발현 국가에 방문한 순으로 집계됐다.

In [ ]:
# 결측치 제거
inf_rsn =  df_patient['infection_reason']
inf_rsn  = inf_rsn.dropna()

In [ ]:
# 워드클라우드로 나타내기
from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in inf_rsn)
wordcloud = WordCloud(max_words = 200, colormap = 'Set3', background_color='white').generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis('off')
plt.figure(1, figsize=(12,12))
plt.show()

### 나이 별 감염 원인 분포 알아보기

In [ ]:
sns.set_style("whitegrid")
sns.FacetGrid(df_patient,  size = 20)\
.map(plt.scatter, 'age', 'infection_reason')\
.add_legend()
plt.title('Age vs Infection Reason',fontsize=30)
plt.xticks(fontsize=12)
plt.yticks(fontsize=15)
plt.show()

다른 국가에 갔다와서 바이러스가 발현된 연령대는 청년층이 중장년층보다 높게 보인다.

### Route Data 살펴보기

In [ ]:
df_route.head()

In [ ]:
# 결측치 체크 하기
df_route.isna().sum() # 결측치 없음 확인

### 경로 데이터를 통한 클러스터링

In [ ]:
# 클러스터링 위한 작업 실시
clus = df_route.loc[:, ['id', 'latitude', 'longitude']]
clus.head()

In [ ]:
# 클러스터 개수 알아보기
K_clusters = range(1,8)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]

Y_axis = df_route[['latitude']]
X_axis = df_route[['longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]

plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.show()

그래프를 볼 때, number of clusters가 3개 이후로 그래프가 완만하기 때문에 최대치인 3개로 설정한다.

In [ ]:
# K 평균 클러스터링 진행
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(clus[clus.columns[1:3]])
clus['cluster_label'] = kmeans.fit_predict(clus[clus.columns[1:3]])
centers = kmeans.cluster_centers_
labels = kmeans.predict(clus[clus.columns[1:3]])

In [ ]:
# 클러스터 시각화
clus.plot.scatter(x = 'latitude', y = 'longitude', c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=100, alpha=0.5)

### *참고 : 전 세계 감염 국가 알아보기 *

In [ ]:
# folium을 통해 지도 시각화
import folium
southkorea_map = folium.Map(location=[36.55,126.983333], zoom_start=7, tiles='Stamen Toner') # 충청도 쯤 부근 위치

for lat, lon, city in zip(df_route['latitude'], df_route['longitude'], df_route['city']):
    folium.CircleMarker([lat, lon],
                      radius = 5,
                      color = 'red',
                      popup = ('City' + str(city) + '<br>'),
                      fill_color = 'red',
                      fill_opacity = 0.7).add_to(southkorea_map)
    
southkorea_map

In [ ]:
# 시간 경과에 따라 퍼지는 정도 지도로 표현
cmap1 = df_route
cmap1 = cmap1.groupby(['date', 'province','latitude','longitude'])['id'].max()


cmap1 = cmap1.reset_index()
cmap1.head()
cmap1['size'] = cmap1['id']*900
cmap1
fig = px.scatter_mapbox(cmap1, lat="latitude", lon="longitude",
                     color="id", size='size',
                     color_continuous_scale='burgyl',
                     animation_frame="date", 
                     title='Spread total cases over time')
fig.update(layout_coloraxis_showscale=True)
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3)
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()


### 지역 별 환자 수 비교하기

In [ ]:
# 시/도 기준 비교
plt.figure(figsize=(15,5))
plt.title('Number patients in province')
df_route.province.value_counts().plot.bar();

In [ ]:
fig = px.pie(values=df_route.groupby(['province']).size().values,names=df_route.groupby(['province']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

### 회귀 모형 만들기

In [ ]:
# setting
data = daily_count.resample('D').first().fillna(0).cumsum()
data = data[20:] # 2/9부터 데이터 사용
x = np.arange(len(data)).reshape(-1, 1)
y = data.values

In [ ]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=[32, 32, 10], max_iter=50000, alpha=0.0005, random_state=26)
_=model.fit(x, y)

In [ ]:
test = np.arange(len(data)+7).reshape(-1, 1)
pred = model.predict(test)
prediction = pred.round().astype(int)
week = [data.index[0] + timedelta(days=i) for i in range(len(prediction))]
dt_idx = pd.DatetimeIndex(week)
predicted_count = pd.Series(prediction, dt_idx)

In [ ]:
# 실제 최근 확진자와 예측 확진자수 그래프로 비교하기
accumulated_count.plot()
predicted_count.plot()
plt.title('Prediction of Accumulated Confirmed Count')
plt.legend(['current confirmd count', 'predicted confirmed count'])
plt.show()

regression model이 실제 데이터를 제대로 반영하여 예측하고 있다.

### Prophet 예언 - 시계열 모델

In [ ]:
prophet= pd.DataFrame(data)
prophet
pr_data = prophet.reset_index()
pr_data.columns = ['ds','y']
pr_data.head()

In [ ]:
# 예측하기
m=Prophet()
m.fit(pr_data)
future=m.make_future_dataframe(periods=15)
forecast=m.predict(future)
forecast

In [ ]:
# 예측 그래프로 나타내기
figure = plot_plotly(m, forecast)
py.iplot(figure) 

figure = m.plot(forecast,xlabel='Date',ylabel='Confirmed Count')

In [ ]:
figure=m.plot_components(forecast)

### ARIMA 시계열 모델

In [ ]:
confirm_cs = pd.DataFrame(data).cumsum()
arima_data = confirm_cs.reset_index()
arima_data.columns = ['confirmed_date','count']
arima_data.head()

In [ ]:
model = ARIMA(arima_data['count'].values, order=(1, 2, 1))
fit_model = model.fit(trend='c', full_output=True, disp=True)
fit_model.summary()

### 예측과 실제 데이터 비교하기

In [ ]:
fit_model.plot_predict()
plt.title('Forecast vs Actual')
pd.DataFrame(fit_model.resid).plot()